In [2]:
from skimage.transform import rotate
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from datetime import datetime
from IPython.display import display, clear_output
from PIL import Image
import pandas as pd
from google.colab import files
import zipfile
import ipywidgets as widgets

PASTA_SAIDA = "imagens_processadas"
os.makedirs(PASTA_SAIDA, exist_ok=True)

historico = []
imagens = []

def upload_imagens_colab():
    """Upload de imagens no Colab e retorna lista de (caminho, imagem)."""
    uploaded = files.upload()
    imgs = []
    for nome in uploaded.keys():
        caminho = nome
        img = cv2.imread(caminho)
        if img is not None:
            imgs.append((caminho, img))
        else:
            print(f"Erro ao carregar {caminho}")
    return imgs

def validar_imagem(img):
    return img is not None and img.shape[0] > 10 and img.shape[1] > 10

def redimensionar_imagem(img, largura_alvo=None, altura_alvo=None):
    h, w = img.shape[:2]
    if largura_alvo and altura_alvo:
        proporcao = min(largura_alvo / w, altura_alvo / h)
        novo_w = int(w * proporcao)
        novo_h = int(h * proporcao)
    elif largura_alvo:
        proporcao = largura_alvo / w
        novo_w = largura_alvo
        novo_h = int(h * proporcao)
    elif altura_alvo:
        proporcao = altura_alvo / h
        novo_w = int(w * proporcao)
        novo_h = altura_alvo
    else:
        return img
    return cv2.resize(img, (novo_w, novo_h), interpolation=cv2.INTER_CUBIC)

def redimensionar_automatico(img, largura_max=800, altura_max=800):
    h, w = img.shape[:2]
    proporcao = min(largura_max / w, altura_max / h, 1.0)
    novo_w, novo_h = int(w * proporcao), int(h * proporcao)
    return cv2.resize(img, (novo_w, novo_h), interpolation=cv2.INTER_CUBIC)

def rotacionar_imagem(img, angulo=0):
    return rotate(img, angle=angulo, resize=True, preserve_range=True).astype(np.uint8)